# KerasCNN2
参考：[A keras prototype](https://www.kaggle.com/knowledgegrappler/a-keras-prototype-0-21174-on-pl)

In [1]:
import numpy as np # linear algebra
np.random.seed(666)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from subprocess import check_output
print(check_output(["ls", "../ShipIceberg/Data"]).decode("utf8"))

sample_submission.csv
sample_submission.csv.7z
test.json
test.json.7z
train.json
train.json.7z



In [2]:
#Load data
train = pd.read_json("../ShipIceberg/Data/train.json")
#test = pd.read_json("../ShipIceberg/Data/test.json")
train.inc_angle = train.inc_angle.replace('na', 0)
train.inc_angle = train.inc_angle.astype(float).fillna(0.0)

#test = pd.read_json("../ShipIceberg/Data/test.json")

In [3]:
import json
with open('../ShipIceberg/Data/test.json', 'r') as f:
    test = json.load(f)
    test=pd.DataFrame(test)
    
print("done!")

done!


In [4]:
# Train data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_train = np.array(train.inc_angle)
y_train = np.array(train["is_iceberg"])

# Test data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_test = np.array(test.inc_angle)


X_train, X_valid, X_angle_train, X_angle_valid, y_train, y_valid = train_test_split(X_train
                    , X_angle_train, y_train, random_state=123, train_size=0.75)

In [5]:
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]

def get_model():
    bn_model = 0
    p_activation = "elu"
    input_1 = Input(shape=(75, 75, 3), name="X_1")
    input_2 = Input(shape=[1], name="angle")
    
    img_1 = Conv2D(16, kernel_size = (3,3), activation=p_activation) ((BatchNormalization(momentum=bn_model))(input_1))
    img_1 = Conv2D(16, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    img_1 = Conv2D(32, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = Conv2D(32, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    img_1 = Conv2D(64, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = Conv2D(64, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    img_1 = Conv2D(128, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    img_1 = GlobalMaxPooling2D() (img_1)
    
    
    img_2 = Conv2D(128, kernel_size = (3,3), activation=p_activation) ((BatchNormalization(momentum=bn_model))(input_1))
    img_2 = MaxPooling2D((2,2)) (img_2)
    img_2 = Dropout(0.2)(img_2)
    img_2 = GlobalMaxPooling2D() (img_2)
    
    img_concat =  (Concatenate()([img_1, img_2, BatchNormalization(momentum=bn_model)(input_2)]))
    
    dense_ayer = Dropout(0.5) (BatchNormalization(momentum=bn_model) ( Dense(256, activation=p_activation)(img_concat) ))
    dense_ayer = Dropout(0.5) (BatchNormalization(momentum=bn_model) ( Dense(64, activation=p_activation)(dense_ayer) ))
    output = Dense(1, activation="sigmoid")(dense_ayer)
    
    model = Model([input_1,input_2],  output)
    optimizer = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

model = get_model()
model.summary()

Using TensorFlow backend.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
X_1 (InputLayer)                 (None, 75, 75, 3)     0                                            
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 75, 75, 3)     12          X_1[0][0]                        
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 73, 73, 16)    448         batch_normalization_1[0][0]      
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 71, 71, 16)    2320        conv2d_1[0][0]                   
___________________________________________________________________________________________

In [6]:
file_path = ".model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=5)

model = get_model()
model.fit([X_train, X_angle_train], y_train, epochs=25, 
          validation_data=([X_valid, X_angle_valid], y_valid), 
          batch_size=32, callbacks=callbacks)

Train on 1203 samples, validate on 401 samples
Epoch 1/25
1203/1203 [==============================] - 24s - loss: 1.1018 - acc: 0.4963 - val_loss: 0.8127 - val_acc: 0.6060
Epoch 2/25
1203/1203 [==============================] - 2s - loss: 0.9721 - acc: 0.5320 - val_loss: 0.8757 - val_acc: 0.6559
Epoch 3/25
1203/1203 [==============================] - 2s - loss: 1.0086 - acc: 0.5303 - val_loss: 1.0176 - val_acc: 0.6384
Epoch 4/25
1203/1203 [==============================] - 2s - loss: 0.8740 - acc: 0.5993 - val_loss: 0.6141 - val_acc: 0.7032
Epoch 5/25
1203/1203 [==============================] - 2s - loss: 0.8863 - acc: 0.5844 - val_loss: 0.9677 - val_acc: 0.6658
Epoch 6/25
1203/1203 [==============================] - 2s - loss: 0.8834 - acc: 0.6185 - val_loss: 0.6327 - val_acc: 0.7207
Epoch 7/25
1203/1203 [==============================] - 3s - loss: 0.8769 - acc: 0.5835 - val_loss: 0.6046 - val_acc: 0.7207
Epoch 8/25
1203/1203 [==============================] - 2s - loss: 0.8315 - a

In [7]:
model.load_weights(filepath=file_path)

print("Train evaluate:")
print(model.evaluate([X_train, X_angle_train], y_train, verbose=1, batch_size=200))
print("####################")
print("watch list evaluate:")
print(model.evaluate([X_valid, X_angle_valid], y_valid, verbose=1, batch_size=200))

Train evaluate:
1203/1203 [==============================] - 1s     
[0.60995252773549691, 0.71487943115872532]
####################
watch list evaluate:
401/401 [==============================] - 0s     
[0.60455677258849438, 0.72069825317496972]


In [8]:
prediction = model.predict([X_test, X_angle_test], verbose=1, batch_size=200)

8424/8424 [==============================] - 4s     


In [9]:
submission = pd.DataFrame({'id': test["id"], 'is_iceberg': prediction.reshape((prediction.shape[0]))})
submission.head(10)

,id,is_iceberg
0,5941774d,0.626411
1,4023181e,0.769256
2,b20200e4,0.191293
3,e7f018bb,0.831009
4,4371c8c3,0.857164
5,a8d9b1fd,0.474142
6,29e7727e,0.818892
7,92a51ffb,0.802191
8,c769ac97,0.527973
9,aee0547d,0.162518


In [10]:
submission.to_csv("./submission.csv", index=False)